SQL:


In [3]:
%load_ext sql
# Connect to SQLite database (creates file if it doesn't exist)
%sql sqlite:///../database/ECOMMERCE.db

Identify Purpose of the System:
1. Why is this db needed?
2. What real-world process we want to digitize?

Therefore, the ecommerce system will need to:
1. Store Product information
2. Allow User registrations
3. Handle Orders

Data to store:
Running an E-commerce will require data like Users, Products, Orders, Payment, Delivery

Identifying Entities:
Entity identification is crucial for database creation. Entities allow us to decide on what tables we need for our database. It is important to focus on picking entities that are independent, logically separable, and posses their own attributes.

Creating tables:
We have to be cautious while creating tables in order to avoid outcomes like storing too much in one table, creating many fragmented tables, queries becoming unnecessarily complex.
For e-commerce website, we need tables such as Users, Products, Orders, Payment, Delivery

In [39]:
%%sql
DROP TABLE IF EXISTS Users;
DROP TABLE IF EXISTS Products;
DROP TABLE IF EXISTS Orders;

CREATE TABLE IF NOT EXISTS Users(
    user_id INTEGER PRIMARY KEY,
    first_name VARCHAR(30) NOT NULL,
    last_name VARCHAR(30) NOT NULL,
    email VARCHAR(50) UNIQUE NOT NULL,
    password VARCHAR(50) NOT NULL,
    created DATETIME DEFAULT CURRENT_TIMESTAMP
);
CREATE TABLE IF NOT EXISTS Products(
    product_id INTEGER PRIMARY KEY,
    name VARCHAR(50) NOT NULL,
    price DECIMAL(10,2) NOT NULL,
    stock INT NOT NULL,
    description TEXT
);
CREATE TABLE IF NOT EXISTS Orders(
    order_id INTEGER PRIMARY KEY,
    user_id INT NOT NULL,
    order_date DATE NOT NULL,
    status TEXT CHECK(status IN ('Placed', 'Processing', 'Shipped', 'Delivered', 'Cancelled')),
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);

CREATE TABLE IF NOT EXISTS OrderItems(
    order_item_id INTEGER PRIMARY KEY,
    order_id INT NOT NULL,
    product_id INT NOT NULL,
    quantity INT NOT NULL,
    unit_price DECIMAL(10,2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);

CREATE TABLE IF NOT EXISTS Payments(
    payment_id INTEGER PRIMARY KEY, 
    order_id INT UNIQUE NOT NULL, 
    payment_date DATETIME DEFAULT CURRENT_TIMESTAMP,
    amount DECIMAL(10,2) NOT NULL,
    method TEXT CHECK(method IN ('Card', 'Cash', 'Esewa', 'FonePay')),
    FOREIGN KEY (order_id) REFERENCES Orders(order_id)
);

CREATE TABLE IF NOT EXISTS Delivery(
    delivery_id INTEGER PRIMARY KEY,
    order_id INT UNIQUE NOT NULL, 
    shipping_address TEXT NOT NULL,
    tracking_number VARCHAR(50) UNIQUE,
    delivery_date DATE,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id)
);

 * sqlite:///../database/ECOMMERCE.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

Identifying attributes:
Users:
    userid,
    first_name,
    last_name,
    email,
    password

Products:
    name,
    price,
    stock,
    description,
    productid

In [40]:
%%sql

INSERT INTO Users(first_name, last_name, email, password) values 
('Ram', 'Chandra', 'exile14years@gmail.com', 'sitasavedsuccessfully'),
('Hari', 'Joshi', 'notharry@gmail.com', 'wishitwasharry');

INSERT INTO Products(name, price, stock, description) values 
('Jacket', 1500, 25, 'Warm, stylish, fur-inside jacket that keeps you warm in winter. Budget-friendly'),
('BLue_Ray Glasses', 1000, 20, 'Protect your eyes with our newly desgined blueray glasses that are clinically tested to put less strain on your eyes.');

INSERT INTO Orders(order_date, status, user_id) values
('2025-05-12', 'Delivered', 1),
('2025-09-05', 'Processing',2);

INSERT INTO OrderItems (order_id, product_id, quantity, unit_price) VALUES
(1, 1, 1, 1500.00), 
(1, 2, 2, 1000.00),
(2, 3, 1, 4500.00);

INSERT INTO Payments (order_id, amount, method) VALUES
(1, 3500.00, 'Card'), 
(2, 4500.00, 'Esewa');

INSERT INTO Delivery (order_id, shipping_address, tracking_number, delivery_date) VALUES
(1, 'Kathmandu, Nepal', 'TKD987654', '2025-11-25'),
(2, 'Lalitpur, Nepal', 'TKD123456', NULL);

 * sqlite:///../database/ECOMMERCE.db
2 rows affected.
2 rows affected.
2 rows affected.
3 rows affected.
(sqlite3.IntegrityError) UNIQUE constraint failed: Payments.order_id
[SQL: INSERT INTO Payments (order_id, amount, method) VALUES
(1, 3500.00, 'Card'), 
(2, 4500.00, 'Esewa');]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


Identifying relationships between entities
Users     Orders	    One-to-Many	    One user can place many orders. Each order belongs to one user.
Orders   Payments	    One-to-One	    One order is generally tied to one payment record (e.g., successful transaction).
Orders   Delivery	    One-to-One	    One order corresponds to one unique shipment/delivery record.
Orders   OrderItems	    One-to-Many	    One order can contain multiple items (rows in OrderItems).
Products  OrderItems	One-to-Many	    One product can appear in many different order items.
Orders   Products	    Many-to-Many	This relationship is resolved by the junction table OrderItems.


Identify Business rules/Constraints

Determine Workflow